In [241]:
import os
import shutil 
import csv
import numpy as np
#import numpy as np
#import pandas as pd
#from sklearn.preprocessing import StandardScaler
from utils import *

In [242]:
band_name = 'v8'  
window_size = 6 

In [265]:
path = 'sources/Ini_sources/'
#print(os.getcwd())
data_1 = make_sequences_y1(band_name, path, band_name)
print(len(data_1))

10


In [266]:
''' Writing out initial sequences
file_ini = open('sources/Ini_sources/sequences_'+band_name, 'w')
for line in data_1:
    file_ini.write(str(line) +'\n')
    
file_ini.close()
'''

" Writing out initial sequences\nfile_ini = open('sources/Ini_sources/sequences_'+band_name, 'w')\nfor line in data_1:\n    file_ini.write(str(line) +'\n')\n    \nfile_ini.close()\n"

In [267]:
from decimal import Decimal, ROUND_HALF_UP, getcontext
getcontext().rounding = ROUND_HALF_UP
l = len(data_1)
h = int(round(Decimal(l)/2,0))
i = int(round(Decimal(h)/2,0))
indexes = [i, l-(h-i)]
print(indexes)

[3, 8]


In [268]:
train = data_1[:indexes[0]] + data_1[indexes[1]:]
test  = data_1[indexes[0]:indexes[1]]
print(len(train))

5


In [71]:
data_test_1 = get_x(test, window_size=window_size)

file_csv = open('sources/test_y1_'+band_name+'.csv', 'w', newline='')
filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6,'Y'])
for i in range(data_test_1.shape[0]):
    filewriter.writerow(data_test_1[i,:])
file_csv.close()

# Creating additional samples for train set

In [269]:
data = get_x(train, window_size=window_size).tolist()
print(len(data))

61


# Negative sequence

In [270]:
data_add = []
for el in data:
    seq = []
    for cell in el[:-1]:
        seq.append(-cell)
    seq.append(1.0)
    data_add.append(seq)

for el in data_add:
    data.append(el)
print(len(data))

122


In [271]:
data_add = []
for el in data:
    el = el[:-1]
    el.reverse()
    el.append(1.)
    data_add.append(el)

for el in data_add:
    data.append(el)
print(len(data))

244


In [272]:
def create_deltas(line):
    deltas = [0]
    for i in range(len(line[:-1])-1):
        delta = line[i] - line[i+1]
        deltas.append(delta)
    return deltas

def create_shift_in_deltas(data, shift):
    data_add = []

    for line in data:
        deltas = create_deltas(line)
        new = [line[0]]
        ini = line[0]
        for i in range(1, len(line[:-1])):
            ini = ini - deltas[i] - shift
            new.append(ini)
        new.append(1.)
        data_add.append(new)
    
    return data_add

In [276]:
shifts = [i for i in np.arange(-0.015, 0.015, 0.0005) if abs(i) > 0.00001]
data_add_res = []
for shift in shifts:
    data_add = create_shift_in_deltas(data, shift)
    data_add_res += data_add
print(len(data_add_res))

#print(data_add_res[2715:2720])

886416
[[3.4878740000000334, 3.4870459999999865, 3.484436000000005, 3.4807840000000394, 3.476576999999997, 3.472749999999905, 1.0], [3.4620459999999866, 3.459436000000005, 3.4557840000000395, 3.4515769999999972, 3.447749999999905, 3.4449750000001127, 1.0], [3.434436000000005, 3.4307840000000396, 3.4265769999999973, 3.422749999999905, 3.419975000000113, 3.418734999999969, 1.0], [3.4057840000000397, 3.4015769999999974, 3.3977499999999052, 3.394975000000113, 3.3937349999999693, 3.392734999999902, 1.0], [3.3765769999999975, 3.3727499999999053, 3.369975000000113, 3.3687349999999694, 3.367734999999902, 3.369011, 1.0]]


In [274]:
data_add = []
for line in data:
    deltas = create_deltas(line)
    #print(line)
    #print('d ', deltas)
    if all(delta <= 0.1 for delta in deltas) is True:
        cell_max = max(map(abs,deltas))
        cell_min = min(map(abs,deltas[1:]))
        while (cell_max < 0.05):
            deltas = [d*10 for d in deltas]
            #print('d ', deltas)
            cell_max *= 10
            new = []
            ini = line[0]
            for i in range(len(deltas)):
                ini = ini - deltas[i]
                new.append(ini)
            new.append(1.)
            #print('n ',new)
            data_add.append(new)
            
            if cell_min >= 0.002:
                deltas2 = [d/2 for d in deltas]
                new = []
                #print('1', deltas)
                #print('2',deltas2)
                ini = line[0]
                for i in range(len(deltas2)):
                    ini = ini - deltas2[i]
                    new.append(ini)
                new.append(1.)
                #print('n ',new)
                data_add.append(new)
            
print(len(data_add))

384


In [275]:
for el in data_add_res:
    data.append(el)

for el in data_add:
    data.append(el)
print(len(data))   

15024


In [25]:
'''
data_add = []
for el in data:
    multiplicate = [2,2.5,3,0.5,0.2,0.1]
    for value in multiplicate:
        line_add = el.copy()
    
        for i in range(len(line_add)-1):
            line_add[i] = round(line_add[i] * value, 3)
        data_add.append(line_add)
        
    increase = [0.5,1,1.5,2,2.5,3,-0.5,-1,-1.5,-2,-2.5,-3]
    for value in increase:
        line_add = el.copy()
    
        for i in range(len(line_add)-1):
            line_add[i] = round(line_add[i] + value, 3)
        data_add.append(line_add)
'''

'\ndata_add = []\nfor el in data:\n    multiplicate = [2,2.5,3,0.5,0.2,0.1]\n    for value in multiplicate:\n        line_add = el.copy()\n    \n        for i in range(len(line_add)-1):\n            line_add[i] = round(line_add[i] * value, 3)\n        data_add.append(line_add)\n        \n    increase = [0.5,1,1.5,2,2.5,3,-0.5,-1,-1.5,-2,-2.5,-3]\n    for value in increase:\n        line_add = el.copy()\n    \n        for i in range(len(line_add)-1):\n            line_add[i] = round(line_add[i] + value, 3)\n        data_add.append(line_add)\n'

In [ ]:
'''
for el in data_add:
    data.append(el)
print(len(data))
'''

# y=0

In [ ]:
def change_one_value_in_sequence(line_ini, delta=0):
    line_add = line_ini.copy()
    line_add[i] = round(line_ini[i-1] + delta, 3)
    line_add[-1] = 0.
    return line_add
    

In [ ]:
data_add = []
for i in range(1, 6):
    for line in data:
        line_add = change_one_value_in_sequence(line, delta=0.002)
        data_add.append(line_add)
        
        line_add = change_one_value_in_sequence(line, delta=-0.002)
        data_add.append(line_add)
        
for line in data_add:
        data.append(line)
        
print(len(data))


# Write out

In [277]:
data_train = np.array(data)

file_csv = open('sources/train_y1_'+band_name+'.csv', 'w', newline='')
filewriter = csv.writer(file_csv, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
filewriter.writerow([1,2,3,4,5,6,'Y'])
for i in range(data_train.shape[0]):
    filewriter.writerow(data_train[i,:])
    '''negative = list(map(lambda x: (-1)*x, data_train_1[i,:-1]))
    negative = np.flip(negative)
    add = data_train_1[i,:].copy()
    add[:-1] = negative
    filewriter.writerow(add)'''
'''for i in range(data_train_1.shape[0]):
    rand = data_train_1[i,:-1].copy()
    np.random.shuffle(rand)
    add = data_train_1[0,:].copy()
    add[:-1] = rand
    add[-1] = 0
    filewriter.writerow(add)
'''
file_csv.close()